In [1]:
import os
from notion_client import Client as NotionClient
from slugify import slugify
import re
import requests
import hashlib
import shutil
import sys

In [2]:
NOTION_TOKEN = os.getenv('NOTION_API_KEY')
NOTION_DATABASE_ID = os.getenv('NOTION_DATABASE_ID')
print(*[NOTION_TOKEN, NOTION_DATABASE_ID], sep='\n')

secret_hzLvMMq9u7qjq2uWW7dNTO2wxADvLI068rLkgy19zP1
f118ba9458f345009bc4564ac6044536


In [3]:
notion = NotionClient(auth=NOTION_TOKEN)
database_id = NOTION_DATABASE_ID

In [4]:
dest_path = os.path.normpath(os.path.abspath(os.path.join('..', 'content', 'blog')))
dest_path

'/Users/er1yaaruma/dev/blog/content/blog'

In [5]:
markdown_pages = {}
regex_meta = re.compile(r'^== *(\w+) *:* (.+) *$')
ignore_root = True

In [7]:
response = notion.search()
from pprint import pprint
pprint(response)

{'has_more': False,
 'next_cursor': None,
 'object': 'list',
 'results': [{'created_time': '2021-05-28T05:32:01.983Z',
              'id': 'f118ba94-58f3-4500-9bc4-564ac6044536',
              'last_edited_time': '2021-05-28T07:51:00.000Z',
              'object': 'database',
              'properties': {'Created': {'created_time': {},
                                         'id': '`_t@',
                                         'type': 'created_time'},
                             'Edited': {'id': 'rRd|',
                                        'last_edited_time': {},
                                        'type': 'last_edited_time'},
                             'Name': {'id': 'title',
                                      'title': {},
                                      'type': 'title'},
                             'Post': {'checkbox': {},
                                      'id': 'g=NP',
                                      'type': 'checkbox'}},
              'title': [{'an

In [8]:
for obj in response['results']:
    if obj['object'] == 'page':
        print("Name: [ {:^50s} ] with ID: [ {:^24s} ]".format(''.join([i['plain_text'] for i in obj['properties']['Name']['title']]), obj['id']))
        blocks = notion.blocks.children.list("d18bf97a-6d1f-4e9d-86d6-0f6577c34eeb")
        for block in blocks['results']:
            if block['type'] in ['paragraph']:
                content = block[block['type']]['text']
                print('    ')
                print(' '.join([item['plain_text'] for item in content]))

Name: [            Catalyst：基於PyTorch的深度學習研發催化劑            ] with ID: [ d18bf97a-6d1f-4e9d-86d6-0f6577c34eeb ]
    
在過去的十幾年當中，隨着深度學習的發展，各式各樣相關的開源項目和框架也逐漸被人所熟知。例如PyTorch就可以說是當中最被研究人員青睞的項目之一。得益於符合Python哲學的設計與良好的底層實現，許多研究團隊都選用Pytorch作爲工具。但是，能力愈強，責任愈大，由於涉及很多與底層密切相關的函數，PyTorch的用戶們常常在研究過程中會遇到不少Bug。另外，隨着硬件加速訓練的崛起，一個能夠在不同硬件平臺上適配的簡單API成爲了當下一個迫切的需要。
    
最近三年，Catalyst-Team一直在研發Catalyst。這是一個基於PyTorch的深度學習研發框架。這個框架致力於改善實驗的可復現性、開發速度以及代碼重用率，因此，用戶們在設計新的實驗時可以把造輪子的時間用來實現一些新的想法。在Catalyst當中，你可以輕易的實現不同的度量函數、模型存檔、多種多樣的日誌方式以及分佈式訓練，並且可以極大避免翻來覆去都差不多的代碼和根本不知所云的底層Bug。
    
在這篇文章當中，我將分享一些我們對於這種高級深度學習框架API的思考，並且利用各種各樣的例子來展示這個項目當前的开发进展。
    
在我們開始之前，我們先來看看在我們用隨機梯度下降法訓練深度學習模型的時候，都在做些什麼：
    
當我們開始一個實驗的時候，可能先定義了一些訓練階段、Epoch數以及數據集的處理。在這之後，我們一遍又一遍的把數據採樣成Batch，然後推理，根據誤差函數算出距離，再利用梯度下降進行參數更新。沒錯，就是這麼簡單。但是，隨着項目的發展和更多Tricks的加入，例如加入了更多不同的誤差函數和更新策略，或者我們要在新的硬件加速平臺上進行訓練，代碼和邏輯可能就不如上面這樣直觀了。
    
爲了解決上面遇到的這類問題，我們創造了Catalyst，一個致力於改善實驗的可復現性、開發速度以及代碼復用的PyTorch深度學習研發框架。這個框架提供了一些在實驗過程當中實用的抽象概念：
    
第一個要介紹的抽象就是 Runner  ，在這個抽象當中，包含了一切你所

In [9]:
notion.blocks.children.list(database_id)

{'object': 'list', 'results': [], 'next_cursor': None, 'has_more': False}

In [10]:
pprint(notion.blocks.children.list("d18bf97a-6d1f-4e9d-86d6-0f6577c34eeb"))

{'has_more': False,
 'next_cursor': None,
 'object': 'list',
 'results': [{'created_time': '2021-05-27T07:41:56.954Z',
              'has_children': False,
              'id': 'd1488745-3b32-4193-9369-1d9254a99372',
              'last_edited_time': '2021-05-28T03:30:00.000Z',
              'object': 'block',
              'type': 'unsupported',
              'unsupported': {}},
             {'created_time': '2021-05-27T06:06:00.000Z',
              'has_children': True,
              'id': '53f7cb95-f4e3-444a-9e3e-cdaf99b05d3c',
              'last_edited_time': '2021-05-27T07:56:00.000Z',
              'object': 'block',
              'type': 'unsupported',
              'unsupported': {}},
             {'created_time': '2021-05-27T05:57:30.669Z',
              'has_children': False,
              'id': 'a5e0f8c4-1cc4-44ae-87eb-60351e1b1168',
              'last_edited_time': '2021-05-27T05:57:00.000Z',
              'object': 'block',
              'paragraph': {'text': [{'annotatio